<a href="https://colab.research.google.com/github/vasanthk99/Data_Science_Project/blob/main/supermarket_sales_prediction_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas, numpy
import os
from IPython.display import display, HTML
import datetime, calendar
from plotly import express
import plotly.graph_objects as go
import matplotlib.pyplot as slspl, seaborn as sestk
from sklearn import metrics, decomposition, model_selection, pipeline,preprocessing, feature_selection, linear_model
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
SlsDf=pandas.read_csv("/content/Amazon Sale Report.csv")
SlsDf.head()

In [ ]:
def ClnCov(sls):
    print("==========================================")
    print("      Data Infomation")
    print("==========================================")
    print(sls.info())
    print("===================================================")
    print("Missing Values Before Cleaning")
    print("===================================================")
    print(sls.isnull().sum())
    smcov=sum(sls.isnull().sum())
    if smcov>0:
        covob=sls.dtypes[sls.dtypes=='object'].index.tolist()
        covnm=sls.dtypes[sls.dtypes!='object'].index.tolist()
        if sum(sls.isna().sum())!=0:
            for x in covob:
                sls[x]=sls[x].fillna(sls[x].mode()[0])
            for y in covnm:
                sls[y]=sls[y].fillna(sls[y].mean())
        print("===================================================")
        print("      Missing Values (After Cleaning)")
        print("===================================================")
        print(sls.isnull().sum())
    else:
        print("===================================================")
        print("      Data has no Missing Values ")
        print("===================================================")
    return sls
SlsDfMS=ClnCov(SlsDf)

In [ ]:
def CovDates(sls):
    sldf="Date"
    slmn=[]
    for x in range(1,13):
        slmn.append(calendar.month_name[x])
    dts_sls=sls[sldf].tolist()
    slmn,slmn1=[],[]
    dates_sls=[]
    year_sls=[]
    idx_months=[]
    new_features=['Year','Days','Month(Num)','Months']
    for d in dts_sls:
        spl=d.split("-")
        year_sls.append(int("20"+spl[2]))
        slmn.append(int(spl[0]))
        slmn1.append(calendar.month_name[int(spl[0])])
        dates_sls.append(int(spl[1]))
    for m in slmn:
        idx_months.append(slmn[m-1])
    try:
        sls.insert(1,new_features[0],numpy.array(year_sls,str))
        sls.insert(2,new_features[1],dates_sls)
        sls.insert(3,new_features[2],idx_months)
        sls.insert(4,new_features[3],slmn1)
    except:
        pass
    #sls=sls.drop('Date',axis=1)
    return sls
SlsDfNew=CovDates(SlsDfMS)
SlsDfNew.head()

EDA


In [ ]:
SlsDfNew['Date'] = pandas.to_datetime(SlsDfNew['Date'])
daily_sales = SlsDfNew.groupby('Date').agg(total_qty=('Qty', 'sum'), total_amount=('Amount', 'sum')).reset_index()
slspl.figure(figsize=(8, 3))
sestk.lineplot(data=daily_sales, x='Date', y='total_qty', label='Total Quantity Sold', color='orange', linewidth=2)
slspl.title('Sales Trends Over Time')
slspl.xlabel('Date')
slspl.ylabel('Sales')
slspl.xticks(rotation=45)
slspl.legend()
slspl.grid(True)
slspl.show()

In [ ]:
RangeIntQ = SlsDfNew['Amount'].quantile(0.75) - SlsDfNew['Amount'].quantile(0.25)
UB = SlsDfNew['Amount'].quantile(0.75) + 1.5*RangeIntQ
LB = SlsDfNew['Amount'].quantile(0.25) - 1.5*RangeIntQ

ValMdn = SlsDfNew['Amount'].median()
SlsDfNew['Amount'] = SlsDfNew['Amount'].apply(lambda x: ValMdn if x > UB  or x < LB else x)
SlsDfNew['Amount'] = SlsDfNew['Amount'].apply(lambda x: ValMdn if x == 0 else x)

fig, my_ax = slspl.subplots(figsize=(7,3))

sestk.histplot(data = SlsDfNew['Amount'], ax=my_ax, binwidth=50, kde=True)
slspl.grid(linestyle='--',color='b')
slspl.title("Distribution of Sales Amount across Count of Orders")
slspl.tight_layout()

In [ ]:
fetssls=['Status','Fulfilment','ship-service-level','Category','Size']
for f in range(len(fetssls)):
    SlsTops=SlsDfNew.groupby(fetssls[f])['Amount'].sum().to_frame().sort_values(by=['Amount'],ascending=False).head(10)
    SlsTops.plot(kind='bar',color='#800080')
    slspl.title('Sales Revenue Amount by {}'.format(fetssls[f]), fontsize=16, weight='bold')
    slspl.ylabel('Total Amount (in millions)', fontsize=14)
    slspl.grid(True, axis='y', linestyle='--', alpha=0.7)
    slspl.xticks(rotation=45, ha="right")
    slspl.legend()
    slspl.tight_layout()
    slspl.show()
    display(HTML(SlsTops.to_html()))

FEATURE ENCODING

In [ ]:
def DataEnc(sls):
    sls1=sls.copy()
    print("Feature Types Before Encoding")
    print(sls1.info())
    obsls=sls1.dtypes[sls1.dtypes=='object'].index.tolist()
    numsls=sls1.dtypes[sls1.dtypes!='object'].index.tolist()
    sls_objdf=sls1[obsls]
    sls_numdf=sls1[numsls]
    cols = sls_objdf.columns.tolist()
    sls_objdf[cols] = sls_objdf[cols].apply(preprocessing.LabelEncoder().fit_transform)
    sls1=pandas.concat([sls_numdf,sls_objdf],axis=1)
    if "Unnamed: 22" in sls1.columns:
        sls1=sls1.drop("Unnamed: 22",axis=1)
    print("Feature Types After Encoding")
    print(sls1.info())
    return sls1
SlsDfNew_encoded=DataEnc(SlsDfNew)
SlsDfNew_encoded.head()

OUTLIER DETECTION

In [ ]:
def OutVarSales(sls,v1):
    arrsls=numpy.array(sls.iloc[:,:-1])
    slspca = decomposition.PCA(n_components=2)
    slspca.fit(arrsls)
    explval=slspca.explained_variance_ratio_
    valm=max(explval.tolist())
    print(explval)
    cmp_slspca=["Comp-{}".format(i+1) for i in range(len(explval.tolist()))]
    slspl.figure(figsize=(5,3))
    valml="%.3f" % valm
    slspl.title("PCA Variance (Max: {})".format(max(explval.tolist())),fontsize=16)
    slspl.bar(cmp_slspca,explval.tolist(),width=0.5,color=v1)
    slspl.plot(explval.tolist(),"--yD")
    slspl.xlabel("Components",fontsize=14)
    slspl.ylabel("PCA Variance",fontsize=14)
    slspl.grid()
    slspl.show()
    return explval
def DataSalScl(sls):
    SCLMM = preprocessing.MinMaxScaler()
    slsscl=SCLMM.fit_transform(sls)
    return slsscl

In [ ]:
vramnt=[]
vramnt.append(OutVarSales(SlsDfNew_encoded.drop(['Date','Amount'],axis=1),"r"))

FlagVar=[]
for vr in vramnt:
    for v in vr:
        if v>0.8:
            FlagVar.append(True)
if len(FlagVar)==1 and True in FlagVar:
    NrmSls=DataSalScl(SlsDfNew_encoded.drop(['Date','Amount'],axis=1))
NrmSls=pandas.DataFrame(NrmSls,columns=SlsDfNew_encoded.drop(['Date','Amount'],axis=1).columns.tolist())
NrmSls['Amount']=SlsDfNew_encoded['Amount'].tolist()
OutVarSales(NrmSls.drop('Amount',axis=1),"c")
NrmSls.head(10)

FEATURE SELECTION

In [ ]:
def Func1(sls,nmst):
    secl="plasma"
    plst=len(sls)
    ttl_text="Correlation for {}".format(nmst)
    '''grpstk.figure(figsize=(14,8))
    grpstk.title(ttl_text,fontsize=25,color="b")
    sestk.heatmap(sls.corr(),fmt="0.2f",cmap=secl,annot=True).
    grpstk.show()'''
    corralldata=pandas.DataFrame(sls.corr()['Amount'])
    corralldata=corralldata.fillna(corralldata.mean())
    feats=corralldata.index.tolist()
    corralldata['Features']=feats
    corralldata.columns=["Coeff","Features"]
    corralldata=corralldata.reset_index(drop=True)
    slscorr=corralldata[(corralldata['Coeff']>0.01)|(corralldata['Coeff']>-0.01)].reset_index(drop=True)
    print("Selected Features Using Correlation: \n",*slscorr.Features.tolist(),sep="\n")
    print("---------------------------------------------------")
    print("     Total Features Selected Using Correlation: {}".format(len(slscorr.Features.tolist())))
    print("---------------------------------------------------")
    return slscorr.Features.tolist()
CorrSel=Func1(NrmSls, "Amazon Sales")

In [ ]:
pred_sls=NrmSls.drop(['Amount'],axis=1)
target_sls=NrmSls['Amount']
pred_sls1=pred_sls
MdRecurr=feature_selection.RFE(estimator=linear_model.LinearRegression(), verbose=1,
                               n_features_to_select=int(len(pred_sls.columns)*0.9),step=0.3)
MdRecurr.fit(pred_sls, target_sls)
print(MdRecurr.ranking_)
MdRecurr.feature_names_in_    #[MdRecurr.support_==True]
rankdf=pandas.DataFrame({"Features":MdRecurr.feature_names_in_,"Rank":MdRecurr.ranking_})
rankdf1=rankdf[rankdf['Rank']==1]
print("Selected Features Using RFE: \n",*rankdf1.Features,sep="\n")
print("--------------------------------------------------------------")
print("---------------------------------------------------")
print("     Total Features Selected Using RFE: {}".format(len(rankdf1.Features)))
print("---------------------------------------------------")
RFESel=rankdf1.Features.tolist()

In [ ]:
Features_Sales=[]
for ff in RFESel:
    if ff in CorrSel:
        Features_Sales.append(ff)

print("Selected Features Using Hybrid Approach (RFE+Correlation): {}".format(len(Features_Sales)))


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas, numpy
import os
from IPython.display import display, HTML
import datetime, calendar
from plotly import express
import plotly.graph_objects as go
import matplotlib.pyplot as slspl, seaborn as sestk
from sklearn import metrics, decomposition, utils, model_selection, pipeline,preprocessing, feature_selection
from sklearn import ensemble, tree, svm, neural_network, linear_model
from statsmodels.tsa.arima.model import ARIMA
import pickle

In [ ]:
NrmSls1=utils.resample(NrmSls,replace = True, n_samples = int(len(NrmSls)*1.5), random_state = 10).reset_index(drop=True)
SlsPred=NrmSls1[Features_Sales]
SlsPred

In [ ]:
SlsAmount=NrmSls1['Amount']
SlsAmount

Data Segmentation


In [ ]:
def SegmentData(prd,tr):
    TrainSlsX,TestSlsX,TrainSlsY,TestSlsY=model_selection.train_test_split(prd,tr, test_size=0.25, random_state=10)
    return TrainSlsX,TestSlsX,TrainSlsY,TestSlsY

TrainSlsX,TestSlsX,TrainSlsY,TestSlsY=SegmentData(SlsPred,SlsAmount)

Function for Sales Prediction

In [ ]:
def SalesPredFunc(rgmd, XSLTR, XSLTS, YSLTR, YSLTS, mdl_pres_sls):
    mdl_pres_sls=rgmd
    mdl_pres_sls.fit(XSLTR, YSLTR)
    prdCov=mdl_pres_sls.predict(XSLTS)
    prdCov_tr=mdl_pres_sls.predict(XSLTR)
    mse=metrics.mean_squared_error(YSLTS,prdCov,squared=True)
    r2nf=round(metrics.r2_score(YSLTS,prdCov),4)*100
    r2nftr=round(metrics.r2_score(YSLTR,prdCov_tr),4)*100
    mapenf=metrics.mean_absolute_percentage_error(YSLTS,prdCov)
    return r2nf,r2nftr,mse,mapenf

## Assigning Models with Default Settings

In [ ]:
ModelSales=[
    tree.DecisionTreeRegressor(),
    linear_model.LinearRegression(),
    svm.LinearSVR(),
    ensemble.RandomForestRegressor(min_weight_fraction_leaf=0.001),
    neural_network.MLPRegressor()
]
ModelNmSales=[
   "Decision Tree",
    "Linear Regression",
    "Support Vector Regression",
    "Random Forest",
    "MLP Regression"
]


## Sales Prediction with Models with Default Settings

In [ ]:
DataSalesPred=[[],[],[],[],[],[]]
for i in range(len(ModelSales)):
    Strt_T = datetime.datetime.now()
    print("Executing {}".format(ModelNmSales[i]))
    PrdCv=SalesPredFunc(ModelSales[i], TrainSlsX, TestSlsX, TrainSlsY, TestSlsY, ModelNmSales[i])
    DataSalesPred[0].append(PrdCv[0])
    DataSalesPred[1].append(PrdCv[1])
    DataSalesPred[2].append(PrdCv[2])
    DataSalesPred[3].append(PrdCv[3])
    EndT = datetime.datetime.now()
    DfTime = EndT-Strt_T
    TimeSec=DfTime.total_seconds()
    DataSalesPred[4].append(round(TimeSec,3))
    print("Elasped Time: {} Seconds".format(round(TimeSec,3)))
    print("______________________________________")

In [ ]:
PredSalesDF=pandas.DataFrame({
    "Model":ModelNmSales,
    "R2_Train(Ex-1)":DataSalesPred[1],
    "R2_Test(Ex-1)":DataSalesPred[0],
    "MSE_Test(Ex-1)":DataSalesPred[2],
    "MAPE_Test(Ex-1)":DataSalesPred[3],
    "Time_Test(Ex-1)":DataSalesPred[4]
})
PredSalesDF

clnf_all_lst=PredSalesDF.columns.tolist()[2:]
for i in clnf_all_lst:
    PredSalesDF=PredSalesDF.sort_values(by=i,ascending=True)
    slspl.figure(figsize=(6,4))
    slspl.title("Comparison of {}".format(i),fontsize=16)
    slspl.barh(PredSalesDF['Model'],PredSalesDF[i],color=["m","b","g","c"])
    slspl.xlabel("Model",fontsize=13)
    slspl.ylabel("{}".format(i),fontsize=13)
    for gid, vlgl in enumerate(PredSalesDF[i]):
        slspl.text(vlgl, gid, str(vlgl))
    slspl.show()
PredSalesDF=PredSalesDF.sort_values(by='R2_Test(Ex-1)',ascending=False).reset_index(drop=True)
PredSalesDF.to_csv("SalesPredictionEx1.csv")
PredSalesDF

## Model Tuning

In [ ]:
ModelSales1=[
    tree.DecisionTreeRegressor(),
    linear_model.LinearRegression(),
    svm.LinearSVR(),
    ensemble.RandomForestRegressor(),
    neural_network.MLPRegressor()
]
TunedModelSales=ModelSales1.copy()
HypTunes=[
    [{"max_depth":[50,60,70],"splitter":["best", "random"]}],
    [{"fit_intercept":[False,True]}],
    [{'tol': [0.01,0.001,0.0001,0.00001],'C':[0.2,0.4,0.6,0.8,1.0],"max_iter":[500,1000,1500]}],
    [{"n_estimators":[50,100],"max_depth":[50,60,70]}],
    [{'solver':['lbfgs','adam'],"alpha":[0.001,0.0001,0.00001],'tol': [0.001,0.0001,0.00001]}]
]
params_tuned, time_tuned= [], []
rgmdls=[
    'DTR.sav',
    'LR.sav',
    'SVR.sav',
    'RFR.sav',
    'MLPR.sav'
]
dirfiles=os.listdir(os.getcwd())
if 'DTR.sav' in dirfiles and 'LR.sav' in dirfiles and 'SVR.sav' in dirfiles and 'RFR.sav' in dirfiles and 'MLPR.sav' in dirfiles:
    print("-----------------------------------------------------")
    print("   Tuned Models are Loading Now ...\n   Loading Model Files")
    print("-----------------------------------------------------")
    rg1=pickle.load(open(rgmdls[0], 'rb'))
    rg2=pickle.load(open(rgmdls[1], 'rb'))
    rg3=pickle.load(open(rgmdls[2], 'rb'))
    rg4=pickle.load(open(rgmdls[3], 'rb'))
    rg5=pickle.load(open(rgmdls[4], 'rb'))
    TunedModelSales=[rg1,rg2,rg3,rg4,rg5]
    print(*TunedModelSales, sep="\n")
else:
    for c in range(len(TunedModelSales)):
        print("____________________________________________________")
        print("Tuning {}".format(ModelNmSales[c]))
        print("____________________________________________________")
        t1 = datetime.datetime.now()
        SlsGrdSrch = model_selection.GridSearchCV(TunedModelSales[c], HypTunes[c], cv = 5, scoring='r2')
        SlsGrdSrch.fit(TrainSlsX, TrainSlsY)
        TunedModelSales[c]=SlsGrdSrch.best_estimator_
        t2 = datetime.datetime.now()
        delta = t2 - t1
        ElapsedTime=delta.total_seconds()
        print("Time Taken for Tuning '{}': {} Seconds".format(ModelNmSales[c],ElapsedTime))
        print("____________________________________________________")
        print("{} Tuning Complete".format(ModelNmSales[c]))
        print("____________________________________________________")
        params_tuned.append(SlsGrdSrch.best_estimator_)
        time_tuned.append(ElapsedTime)
    for hm in range(len(TunedModelSales)):
        pickle.dump(TunedModelSales[hm], open(rgmdls[hm], 'wb'))
    TuningData=pandas.DataFrame({"Classifier":ModelNmSales,"Tuned_Parameters":params_tuned,"Tuning_Time(Sec)":time_tuned})
    #display(HTML(TuningData.to_html()))

____________________________________________________
Tuning Decision Tree
____________________________________________________
Time Taken for Tuning 'Decision Tree': 35.33365 Seconds
____________________________________________________
Decision Tree Tuning Complete
____________________________________________________
____________________________________________________
Tuning Linear Regression
____________________________________________________
Time Taken for Tuning 'Linear Regression': 1.314008 Seconds
____________________________________________________
Linear Regression Tuning Complete
____________________________________________________
____________________________________________________
Tuning Support Vector Regression
____________________________________________________
Time Taken for Tuning 'Support Vector Regression': 68.675929 Seconds
____________________________________________________
Support Vector Regression Tuning Complete
_______________________________________________